In [1]:
import pandas as pd
import numpy as np
import glob as glob
import json
import sklearn
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from random import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, explained_variance_score


import keras
from keras.layers import Dense, Dropout, RepeatVector, BatchNormalization, Convolution1D, Flatten, Lambda, Permute, MaxPooling1D, AlphaDropout
from keras.models import Sequential
from keras.utils import to_categorical
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.preprocessing import StandardScaler


Using TensorFlow backend.


In [3]:
root_path = 'features_by_valence/'
label_set = ['low', 'high']
low_files = glob.glob(root_path + label_set[0] + '/*')
high_files = glob.glob(root_path + label_set[1] + '/*')

all_features = []

for filename in low_files:
    word_histogram = json.loads(open(filename).read())
    all_features.append({
        'dictionary': word_histogram, 
        'filename': filename.split('/')[-1]})
for filename in high_files:
    word_histogram = json.loads(open(filename).read())
    all_features.append({
        'dictionary': word_histogram, 
        'filename': filename.split('/')[-1]})

In [14]:
root_path = 'stock_price_changes_by_ticker_and_date/'

# filenames = [f['filename'] for f in all_features]
for i, f in enumerate(all_features):
    pc = float(open(root_path + f['filename']).read())
    all_features[i]['percent_change'] = pc

In [17]:
n_total = int(len(all_features))
n_train = int(n_total * 0.8)
n_test = int(n_total * 0.2)

print('total: ' + str(n_total))
print('train: ' + str(n_train))
print('test: ' + str(n_test))

total: 252
train: 201
test: 50


In [18]:
# one hot encoding

def word_list(percent_occurance):
    key_set = {}

    for feature_d in all_features:
        keys = feature_d['dictionary'].keys()
        for key in keys:
            key_set[key] = key_set.get(key, 0) + 1
    key_set
    # Return words that occur in the top percent_occurance% of documents
    key_set = dict([[key, key_set[key]] for key in key_set if (key_set[key]/float(n_total) > float(percent_occurance))])

    words_sorted_alphabetically = list(key_set.keys())
    words_sorted_alphabetically.sort()
    return words_sorted_alphabetically

In [19]:
def get_index(reference, key):
    try:
        return reference.index(key)
    except:
        return len(reference)

In [20]:
ohe = {}
i_2_label = {}

def one_hot_encode(reference, dictionary):
    ohe = np.zeros(len(reference) + 1)
    for key in dictionary.keys():
        pos = get_index(reference, key)
        ohe[pos] = dictionary[key]
    norm_lookup = float(np.linalg.norm(ohe))
    normalized_ohe = [float(item)/float(norm_lookup) for item in ohe]
    return normalized_ohe

In [21]:
def preds_and_true(clf, af, ohe_words):
    correct_preds = 0
    all_preds = 0
    preds = []
    true = []
    for d_item in af:
        feature = d_item['ohe_feature']
        label = d_item['percent_change']
        classification = clf.predict([feature])[0]
        preds.append(classification)
        true.append( label )
    return [preds, true]

In [22]:
def encode_features_and_words(all_features, word_threshold):
    ohe_words = word_list(word_threshold)
    
    for i, f in enumerate(all_features):
        ohe_feature = one_hot_encode(ohe_words, f['dictionary'])
        all_features[i]['ohe_feature'] = ohe_feature
    return [all_features, ohe_words]


In [23]:
# n = 0.001
len(word_list(0.25))

2822

In [25]:
def get_train_test(all_features, n=0.25):
    all_features, ohe_words = encode_features_and_words(all_features, n)

    shuffle(all_features)

    train_features = all_features[0:n_train]
    test_features = all_features[n_train:n_total]

    X = [f['ohe_feature'] for f in train_features]
    y = [f['percent_change'] for f in train_features]
    
    x_val = [f['ohe_feature'] for f in test_features]
    y_val = [f['percent_change'] for f in test_features]
    return [X, y, x_val, y_val]

In [90]:
# 0.25 is a solid value for n. Lower values mean letting in larger amounts of words (features)
# Once validation accuracy is 66.67%, save the weights. That's the max. 
n = 0.25
X, y, x_val, y_val = get_train_test(all_features, n)

scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)
x_val = scaler.transform(x_val)

In [27]:
shape = len(word_list(n)) + 1
shape

2823

In [103]:
shape = len(word_list(n)) + 1

m1 = Sequential([
    BatchNormalization(input_shape=(shape,)),
    Dense(80, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(40, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(20, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(1, activation='sigmoid')   
])

m1.compile(optimizer='rmsprop',
              loss='mean_absolute_error',
              metrics=['mae'])

sched = [[0.0001, 2], [0.001, 20], [0.01, 2], [0.1, 2], [0.5, 1], [0.1, 5], [0.01, 20], [0.001, 40], [0.0001, 80], [0.00005, 120]]

for i in range(5):
    for info in sched:
        lr, epochs = info
        m1.optimizer.lr = lr
        m1.fit(np.array(X), np.array(y), epochs=epochs,  batch_size=64, validation_data=(np.array(x_val), np.array(y_val)))

Train on 201 samples, validate on 51 samples
Epoch 1/2
201/201 [==============================] - 2s 8ms/step - loss: 0.5015 - mean_absolute_error: 0.5015 - val_loss: 0.3349 - val_mean_absolute_error: 0.3349
Epoch 2/2
201/201 [==============================] - 0s 449us/step - loss: 0.5002 - mean_absolute_error: 0.5002 - val_loss: 0.3466 - val_mean_absolute_error: 0.3466
Train on 201 samples, validate on 51 samples
Epoch 1/20
201/201 [==============================] - 0s 528us/step - loss: 0.5019 - mean_absolute_error: 0.5019 - val_loss: 0.3548 - val_mean_absolute_error: 0.3548
Epoch 2/20
201/201 [==============================] - 0s 360us/step - loss: 0.5025 - mean_absolute_error: 0.5025 - val_loss: 0.3568 - val_mean_absolute_error: 0.3568
Epoch 3/20
201/201 [==============================] - 0s 470us/step - loss: 0.4985 - mean_absolute_error: 0.4985 - val_loss: 0.3634 - val_mean_absolute_error: 0.3634
Epoch 4/20
201/201 [==============================] - 0s 493us/step - loss: 0.4957 -

Epoch 5/80
201/201 [==============================] - 0s 396us/step - loss: 0.4485 - mean_absolute_error: 0.4485 - val_loss: 0.4609 - val_mean_absolute_error: 0.4609
Epoch 6/80
201/201 [==============================] - 0s 511us/step - loss: 0.4502 - mean_absolute_error: 0.4502 - val_loss: 0.4619 - val_mean_absolute_error: 0.4619
Epoch 7/80
201/201 [==============================] - 0s 815us/step - loss: 0.4473 - mean_absolute_error: 0.4473 - val_loss: 0.4607 - val_mean_absolute_error: 0.4607
Epoch 8/80
201/201 [==============================] - 0s 665us/step - loss: 0.4461 - mean_absolute_error: 0.4461 - val_loss: 0.4572 - val_mean_absolute_error: 0.4572
Epoch 9/80
201/201 [==============================] - 0s 765us/step - loss: 0.4418 - mean_absolute_error: 0.4418 - val_loss: 0.4586 - val_mean_absolute_error: 0.4586
Epoch 10/80
201/201 [==============================] - 0s 386us/step - loss: 0.4437 - mean_absolute_error: 0.4437 - val_loss: 0.4599 - val_mean_absolute_error: 0.4599
Epo

Epoch 23/120
201/201 [==============================] - 0s 368us/step - loss: 0.3848 - mean_absolute_error: 0.3848 - val_loss: 0.3793 - val_mean_absolute_error: 0.3793
Epoch 24/120
201/201 [==============================] - 0s 313us/step - loss: 0.3865 - mean_absolute_error: 0.3865 - val_loss: 0.3789 - val_mean_absolute_error: 0.3789
Epoch 25/120
201/201 [==============================] - 0s 334us/step - loss: 0.3961 - mean_absolute_error: 0.3961 - val_loss: 0.3775 - val_mean_absolute_error: 0.3775
Epoch 26/120
201/201 [==============================] - 0s 411us/step - loss: 0.3938 - mean_absolute_error: 0.3938 - val_loss: 0.3748 - val_mean_absolute_error: 0.3748
Epoch 27/120
201/201 [==============================] - 0s 401us/step - loss: 0.3946 - mean_absolute_error: 0.3946 - val_loss: 0.3749 - val_mean_absolute_error: 0.3749
Epoch 28/120
201/201 [==============================] - 0s 396us/step - loss: 0.3869 - mean_absolute_error: 0.3869 - val_loss: 0.3739 - val_mean_absolute_error:

Epoch 72/120
201/201 [==============================] - 0s 323us/step - loss: 0.3623 - mean_absolute_error: 0.3623 - val_loss: 0.3424 - val_mean_absolute_error: 0.3424
Epoch 73/120
201/201 [==============================] - 0s 322us/step - loss: 0.3563 - mean_absolute_error: 0.3563 - val_loss: 0.3425 - val_mean_absolute_error: 0.3425
Epoch 74/120
201/201 [==============================] - 0s 368us/step - loss: 0.3658 - mean_absolute_error: 0.3658 - val_loss: 0.3426 - val_mean_absolute_error: 0.3426
Epoch 75/120
201/201 [==============================] - 0s 360us/step - loss: 0.3670 - mean_absolute_error: 0.3670 - val_loss: 0.3432 - val_mean_absolute_error: 0.3432
Epoch 76/120
201/201 [==============================] - 0s 443us/step - loss: 0.3626 - mean_absolute_error: 0.3626 - val_loss: 0.3451 - val_mean_absolute_error: 0.3451
Epoch 77/120
201/201 [==============================] - 0s 390us/step - loss: 0.3557 - mean_absolute_error: 0.3557 - val_loss: 0.3429 - val_mean_absolute_error:

201/201 [==============================] - 0s 479us/step - loss: 0.3056 - mean_absolute_error: 0.3056 - val_loss: 0.2431 - val_mean_absolute_error: 0.2431
Epoch 16/20
201/201 [==============================] - 0s 420us/step - loss: 0.3099 - mean_absolute_error: 0.3099 - val_loss: 0.2429 - val_mean_absolute_error: 0.2429
Epoch 17/20
201/201 [==============================] - 0s 430us/step - loss: 0.2987 - mean_absolute_error: 0.2987 - val_loss: 0.2414 - val_mean_absolute_error: 0.2414
Epoch 18/20
201/201 [==============================] - 0s 524us/step - loss: 0.3103 - mean_absolute_error: 0.3103 - val_loss: 0.2422 - val_mean_absolute_error: 0.2422
Epoch 19/20
201/201 [==============================] - 0s 537us/step - loss: 0.3022 - mean_absolute_error: 0.3022 - val_loss: 0.2427 - val_mean_absolute_error: 0.2427
Epoch 20/20
201/201 [==============================] - 0s 456us/step - loss: 0.2903 - mean_absolute_error: 0.2903 - val_loss: 0.2425 - val_mean_absolute_error: 0.2425
Train on 2

...

Epoch 118/120
201/201 [==============================] - 0s 302us/step - loss: 0.0434 - mean_absolute_error: 0.0434 - val_loss: 0.0282 - val_mean_absolute_error: 0.0282
Epoch 119/120
201/201 [==============================] - 0s 304us/step - loss: 0.0431 - mean_absolute_error: 0.0431 - val_loss: 0.0282 - val_mean_absolute_error: 0.0282
Epoch 120/120
201/201 [==============================] - 0s 303us/step - loss: 0.0382 - mean_absolute_error: 0.0382 - val_loss: 0.0282 - val_mean_absolute_error: 0.0282
Train on 201 samples, validate on 51 samples
Epoch 1/2
201/201 [==============================] - 0s 303us/step - loss: 0.0425 - mean_absolute_error: 0.0425 - val_loss: 0.0282 - val_mean_absolute_error: 0.0282
Epoch 2/2
201/201 [==============================] - 0s 315us/step - loss: 0.0431 - mean_absolute_error: 0.0431 - val_loss: 0.0282 - val_mean_absolute_error: 0.0282
Train on 201 samples, validate on 51 samples
Epoch 1/20
201/201 [==============================] - 0s 554us/step - los

Epoch 2/80
201/201 [==============================] - 0s 288us/step - loss: 0.0394 - mean_absolute_error: 0.0394 - val_loss: 0.0278 - val_mean_absolute_error: 0.0278
Epoch 3/80
201/201 [==============================] - 0s 283us/step - loss: 0.0397 - mean_absolute_error: 0.0397 - val_loss: 0.0278 - val_mean_absolute_error: 0.0278
Epoch 4/80
201/201 [==============================] - 0s 308us/step - loss: 0.0386 - mean_absolute_error: 0.0386 - val_loss: 0.0278 - val_mean_absolute_error: 0.0278
Epoch 5/80
201/201 [==============================] - 0s 302us/step - loss: 0.0452 - mean_absolute_error: 0.0452 - val_loss: 0.0278 - val_mean_absolute_error: 0.0278
Epoch 6/80
201/201 [==============================] - 0s 295us/step - loss: 0.0459 - mean_absolute_error: 0.0459 - val_loss: 0.0278 - val_mean_absolute_error: 0.0278
Epoch 7/80
201/201 [==============================] - 0s 275us/step - loss: 0.0414 - mean_absolute_error: 0.0414 - val_loss: 0.0278 - val_mean_absolute_error: 0.0278
Epoc

201/201 [==============================] - 0s 281us/step - loss: 0.0361 - mean_absolute_error: 0.0361 - val_loss: 0.0277 - val_mean_absolute_error: 0.0277
Epoch 20/120
201/201 [==============================] - 0s 290us/step - loss: 0.0373 - mean_absolute_error: 0.0373 - val_loss: 0.0277 - val_mean_absolute_error: 0.0277
Epoch 21/120
201/201 [==============================] - 0s 286us/step - loss: 0.0411 - mean_absolute_error: 0.0411 - val_loss: 0.0277 - val_mean_absolute_error: 0.0277
Epoch 22/120
201/201 [==============================] - 0s 282us/step - loss: 0.0375 - mean_absolute_error: 0.0375 - val_loss: 0.0277 - val_mean_absolute_error: 0.0277
Epoch 23/120
201/201 [==============================] - 0s 282us/step - loss: 0.0372 - mean_absolute_error: 0.0372 - val_loss: 0.0277 - val_mean_absolute_error: 0.0277
Epoch 24/120
201/201 [==============================] - 0s 275us/step - loss: 0.0375 - mean_absolute_error: 0.0375 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch

Epoch 68/120
201/201 [==============================] - 0s 287us/step - loss: 0.0410 - mean_absolute_error: 0.0410 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch 69/120
201/201 [==============================] - 0s 282us/step - loss: 0.0378 - mean_absolute_error: 0.0378 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch 70/120
201/201 [==============================] - 0s 304us/step - loss: 0.0388 - mean_absolute_error: 0.0388 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch 71/120
201/201 [==============================] - 0s 303us/step - loss: 0.0414 - mean_absolute_error: 0.0414 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch 72/120
201/201 [==============================] - 0s 310us/step - loss: 0.0376 - mean_absolute_error: 0.0376 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch 73/120
201/201 [==============================] - 0s 307us/step - loss: 0.0361 - mean_absolute_error: 0.0361 - val_loss: 0.0276 - val_mean_absolute_error:

Epoch 117/120
201/201 [==============================] - 0s 315us/step - loss: 0.0372 - mean_absolute_error: 0.0372 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch 118/120
201/201 [==============================] - 0s 312us/step - loss: 0.0398 - mean_absolute_error: 0.0398 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch 119/120
201/201 [==============================] - 0s 313us/step - loss: 0.0371 - mean_absolute_error: 0.0371 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276
Epoch 120/120
201/201 [==============================] - 0s 301us/step - loss: 0.0390 - mean_absolute_error: 0.0390 - val_loss: 0.0276 - val_mean_absolute_error: 0.0276


array([[ 0.00646421]], dtype=float32)

In [63]:
for i in range(len(X)):
    print(m1.predict(np.array([X[i]])))
    print(y[i])
    print("\n")

[[-0.03393828]]
-0.0005292405398252453


[[-0.03410605]]
0.008333333333333255


[[-0.01393568]]
-0.04828973843058361


[[-0.05682974]]
0.008258680999914739


[[-0.06315508]]
-0.03969128996692402


[[ 0.0414346]]
-0.010504201680672232


[[-0.07775185]]
0.00330429558425952


[[-0.01503236]]
-0.017325680272108797


[[-0.05437827]]
0.0689941812136326


[[-0.01032812]]
-0.004714757190004782


[[ 0.03748613]]
0.11038961038961026


[[ 0.03277944]]
0.06974477958236659


[[ 0.0618001]]
0.060831509846826996


[[-0.00046651]]
-0.018439716312056723


[[-0.0925583]]
-0.019653179190751477


[[-0.02209948]]
0.017944535073409516


[[ 0.01048759]]
0.009907755380936084


[[ 0.03909699]]
0.03676012461059189


[[-0.05168632]]
0.02627257799671595


[[-0.00342466]]
-0.02448210922787202


[[ 0.05398517]]
0.05106382978723409


[[ 0.00389128]]
-0.021212121212121165


[[ 0.01657477]]
0.3072440944881889


[[ 0.0322951]]
0.040425531914893585


[[-0.06094308]]
-0.06740389678778293


[[ 0.00656767]]
0.0138888888888

In [11]:
import numpy as np
# np.average()
X = [[1,2],[3,4]]
np.sum(X, axis=0)/len(X)

array([ 2.,  3.])

In [25]:
preds = [-0.01, 0.21, 0.004, -0.42]
true = [-0.11, -0.01, 0.2, 0.15]
score = 0
indices = []
for i, pred in enumerate(np.array(preds) > 0):
    if pred == True:
        indices.append(i)
for i in indices:
    score += true[i]
score

0.19

In [19]:
pos

[0, 1, 2, 3]

In [34]:
import glob
len(glob.glob('../../stanford_nlp_sec/8K-gz/*'))

1500